In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(precision=3)


data = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/jester-data-1.csv",header=None)

In [ ]:

data = data.drop([data.columns[0]], axis=1)
data = data.replace(99, np.NaN)

In [ ]:
data = data.drop([data.columns[0]], axis=1)

In [ ]:
datamean = data.mean(axis=0)
print(datamean)

1      0.904573
2      0.208460
3      0.316574
4     -1.449114
5      0.403556
         ...   
96     1.528003
97     1.673750
98     0.767692
99    -0.031244
100    1.354711
Length: 100, dtype: float64


In [ ]:
datamean.min()

-3.8338796373689923

In [ ]:
datamean.max()

3.6650848950824937

In [ ]:
max_index = datamean.idxmax() 
print(max_index)

50


In [ ]:
arr = data.values
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

2498300 (2498300,)


In [ ]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [ ]:
new_arr, idx = replace(arr, 0.1) #validation set

In [ ]:


arr[idx[0][0], idx[1][0]]

1.12

In [ ]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [ ]:
n_latent_factors = 2

user_ratings = data.values
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [ ]:
latent_user_preferences

array([[0.384, 0.394],
       [0.34 , 0.226],
       [0.443, 0.187],
       ...,
       [0.603, 0.519],
       [0.695, 0.402],
       [0.248, 0.119]])

In [ ]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    #print(item_id)
    predicted_rating = predict_rating(user_id, item_id)
    err =  predicted_rating - rating
    #print(err)
    user_pref_values = latent_user_preferences[user_id]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    mse_history = []
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id, item_id]
                if not np.isnan(rating) and rating != 99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 1) == 0:    #since we need to be shown for each interation, we want to get for each one the mse_history, and then the appended list. 
            print('Iteration %d/%d:\tMSE=%.6f' % (iteration, iterations, mse))
            mse_history.append(mse)
    return mse_history

In [ ]:
num_iter = 10
hist = sgd(num_iter)  # Note how the MSE decreases with the number of iterations
plt.figure()
plt.plot(np.arange(0, num_iter, 1), hist)
plt.xlabel("Iterations")
plt.ylabel("MSE")
plt.show()

Iteration 0/10:	MSE=23.022359
Iteration 1/10:	MSE=22.797836
Iteration 2/10:	MSE=22.580571
Iteration 3/10:	MSE=22.371401
Iteration 4/10:	MSE=22.170750
Iteration 5/10:	MSE=21.978578
Iteration 6/10:	MSE=21.794333
Iteration 7/10:	MSE=21.616905


In [ ]:
data.iloc[573, 50]

-4.42

In [ ]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

In [ ]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = data.columns